In [5]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import os, datetime

## Script Objective
 - Read in files determining which wells will represent which groups in a given scenario & what fraction of the total group volume they will use
 - Proportionally distribute injection or extraction volumes amongst wells specified as active for a given scenario
 

### Well Groups
There are 5 wells groups that are assigned different volumes of extraction or injection.
1. LVL Well: Injection
2. Alamitos Barrier: Injection
3. New Well Field: Injection for LB ATW
4. New Well Field: Injection for LC ATW
5. Wasted to SGR/Available Addtl AW flows: Injection
<br>
There needs to be Extraction linked to wherever there is injection that counts towards augmentation

In [2]:
scenario = 'test'

In [19]:
fraction_data = pd.read_excel('input/scenario_inputs.xlsx', sheet_name='Fraction_Active_Data')
new_CLNs = fraction_data.CLN.drop_duplicates().values
new_CLNs = np.delete(new_CLNs,new_CLNs=="alamitos")

array([1793], dtype=object)

In [4]:
if fraction_data.loc["alamitos"] > 0:
    alamitos_extra_inj = True

,CLN,name,group,type,fraction
0,1793,CB-SW-1,LVL,Injection,1


In [9]:
USGS_wells = gpd.read_file("S:/LAX/JACOBS.C001.CNSLT/GIS/Task3_USGmodeling/shp/2_lab_ug.cln_Item8_GWnodes_WellGroups.shp")
alamitos_wells = USGS_wells[USGS_wells.group=='Alamitos Barrier_Injection']

In [17]:
alamitos_CLN = alamitos_wells.IFNOD.drop_duplicates().values
alamitos_df = pd.DataFrame(1, index = alamitos_CLN, columns=["default"])

In [16]:
base_WEL = open("S:/LAX/DUDEK.C001.SMBGSA/INTERA models/UpdatedModel/ModelWithNewExe/input/lab_ug_Corrected_Updated.wel", "r")
new_WEL  = open("S:/LAX/JACOBS.C001.CNSLT/Phase-2/LVL/4000.MODELS/test/lab_ug_test.wel", "w")
new_WEL.write(base_CLN.readline())

NameError: name 'new_CLN' is not defined

In [ ]:
# need to run through this one time to get total Alamitos volumes for baseline
# goal is to have dataframe of each group's fluxes by stress period
for sp in range(1,181):
    SPHeader = base_CLN.readline()
    ITMP, NP, ITMPCLN = int(SPHeader.split()[0]), int(SPHeader.split()[1]), int(SPHeader.split()[2])
    if sp<61:
        new_WEL.write(SPHeader)
        for u in range(ITMP + ITMPCLN):
            new_WEL.write(base_CLN.readline())
    else:
        
        new_ITMPCLN = ITMPCLN + len(new_CLNs)
        new_WEL.write("%d %d %d  #Stress Period %d\n" % (ITMP, NP, new_ITMPCLN, sp))
        for u in range(ITMP):
            new_CLN.write(base_CLN.readline())
        for c in range(ITMPCLN):
            CLN_row = base_CLN.readline()
            CLN_No = int(CLN_row.split()[0])
            flux = float(CLN_row.split()[1])
            alamitos_df["SP%d_flux"%sp] = 0
            if CLN_No in alamitos_CLN and alamitos_extra_inj:
                alamitos_df.loc[CLN_No,"SP%d_flux"%sp] = flux
                Extra_alamitos_vol = scenario_fluxes.loc[sp,"Scenario%d"%scenario]
            else:
                new_WEL.write(CLN_row)